<a href="https://colab.research.google.com/github/fedoseev070888/HW/blob/main/save_weights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense, Flatten
from tensorflow.keras.optimizers import Adam
import os

# **Создаём основную модель для обучения и предсказаний**

In [ ]:
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
vgg16_net.trainable = False

model = Sequential()
model.add(vgg16_net)
model.add(Flatten())
model.add(Dense(4096, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 4096)              102764544 
                                                                 
 dense_1 (Dense)             (None, 1)                 4097      
                                                                 
Total params: 117,483,329
Trainable params: 102,768,641
Non-trainable params: 14,714,688
_________________________________________________________________


# **Обучаем**

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [ ]:
#model.fit_generator(train_ds, validation_data = validation_ds, shuffle=True, epochs=20, verbose=1)

# **Создаём модель для сохранения части весов основной модели**

In [ ]:
neuron_count = model.layers[2].get_weights()[0].shape[1] # количество нейронов в предпоследнем слое основной модели
input = model.layers[2].get_weights()[0].shape[0]        # количество связей предпоследнего слоя с предыдущим слоем в основной модели


model_for_save = Sequential()
model_for_save.add(Dense(neuron_count, input_dim = input))
model_for_save.add(Dense(1))
model_for_save.build()

# **Присваиваем веса в model_for_save из model**

In [ ]:
w_1 = model.layers[2].get_weights()[0]              # веса-множители для предпоследнего слоя для 4096 нейронов 
b_1 = model.layers[2].get_weights()[1]              # смещение для предпоследнего слоя для 4096 нейронов 

w_2 = model.layers[3].get_weights()[0]              # веса-множители для последнего слоя для 1 нейрона
b_2 = model.layers[3].get_weights()[1]              # смещение для последнего слоя для 1 нейрона

model_for_save.layers[0].set_weights([w_1, b_1])    # присваиваем веса предпоследнего слоя
model_for_save.layers[1].set_weights([w_2, b_2])    # присваиваем веса последнего слоя

# **Сохраняем веса**

In [ ]:
model_for_save.save_weights('model_for_save.h5')

# **Загружаем веса**

In [ ]:
model_for_save.load_weights('model_for_save.h5')

# **Присваиваем веса обратно в model из model_for_save**

In [ ]:
w_1 = model_for_save.layers[0].get_weights()[0]
b_1 = model_for_save.layers[0].get_weights()[1]

w_2 = model_for_save.layers[1].get_weights()[0]
b_2 = model_for_save.layers[1].get_weights()[1]

model.layers[2].set_weights([w_1, b_1])
model.layers[3].set_weights([w_2, b_2])

# **Сравним размер фалов весов обоих моделей**

рзмер файла весов **model_for_save**

In [ ]:
os.path.getsize("/content/model_for_save.h5")

411086852

рзмер файла весов основной модели **model**

In [ ]:
model.save_weights('model.h5')

In [ ]:
os.path.getsize("/content/model.h5")

469969824

рзмер файла весов основной модели **model**, но без полносвязных слоёв

In [ ]:
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
vgg16_net.trainable = False

model = Sequential()
model.add(vgg16_net)

In [ ]:
model.save_weights('model_without_top.h5')

In [ ]:
os.path.getsize("/content/model_without_top.h5")

58888264